In [1]:
# copied from last notebook 
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Importing required libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import sklearn
import matplotlib.pyplot as plt

from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Input

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.image import load_img


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# get data ready
y_train = pd.read_csv("/home/mitchell/kaggleData/dogBreedIdentifier/data/labels.csv")

train_image_dir = "/home/mitchell/kaggleData/dogBreedIdentifier/data/train/"
test_image_dir = "/home/mitchell/kaggleData/dogBreedIdentifier/data/test/"

df_test = pd.read_csv('/home/mitchell/kaggleData/dogBreedIdentifier/data/sample_submission.csv')

In [3]:
# view the csv loaded
y_train.head()


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
y_train.columns

Index(['id', 'breed'], dtype='object')

In [5]:
#one hot envcoding of the labesl
breed = pd.Series(y_train['breed'])
breed
one_hot = pd.get_dummies(breed, sparse = True)
#one_hot

In [6]:
one_hot_labels = np.asarray(one_hot)
one_hot_labels.shape


(10222, 120)

In [7]:
#create pivto table then plot to see the distribution of the training set
y_plot=pd.pivot_table(y_train, index='breed', aggfunc=len)
y_plot = y_plot.sort_values('id', ascending=False)
#y_plot
#y_plot.head()

In [8]:
#using: 'ls' > trainFilenames.csv 
# in terminal, created a csv with all filenames.

trainFilenames = pd.read_csv("/home/mitchell/kaggleData/dogBreedIdentifier/data/trainFilenames.csv", header=None)
trainFilenames.size

10222

In [9]:
# this line reformats the data into numpy array
trainFilenames=trainFilenames.T.as_matrix()
trainFilenames=trainFilenames.tolist()
#trainFilenames

In [10]:
#list comprehension trick to take list out of a list..
trainFilenames = [y for x in trainFilenames for y in x]
#trainFilenames

In [11]:
# we can use this to join 

train_img_paths = [train_image_dir + s for s in trainFilenames]
#train_img_paths

In [12]:
# initialize lists and variables
x_train = []
train_labels = []
x_test = []
im_size = 128
img_height=im_size
img_width=im_size
bs = 32

In [24]:
i = 0 
for f, breed in tqdm(y_train.values):
    
    img = load_img(train_image_dir + '{}.jpg'.format(f), target_size=(img_height, img_width))
    #before image is appended it must be converted into array
    img = np.array(img, dtype="int32")
    x_train.append(img)
    
    
    label = one_hot_labels[i]
    train_labels.append(label)
    i += 1


100%|██████████| 10222/10222 [00:13<00:00, 755.76it/s]


In [14]:
for f in tqdm(df_test['id'].values):
    
    img = load_img(test_image_dir + '{}.jpg'.format(f), target_size=(img_height, img_width))
    img = np.array(img, dtype="int32")
    x_test.append(img)

100%|██████████| 10357/10357 [00:20<00:00, 494.84it/s]


In [25]:
#these few lines reformat the data
y_train_raw = np.array(train_labels, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

In [26]:
#x_train_raw_min = x_train_raw[:9000,:,:,:]

In [27]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(9000, 128, 128, 3)
(20444, 120)
(10357, 128, 128, 3)


In [17]:
#We can see above that there are 120 different breeds. 
#We can put this in a num_class variable below that can then be used when creating the CNN model.
num_class = y_train_raw.shape[1]

In [18]:
# build a croos val set
X_train, X_val, Y_train, Y_val = train_test_split(x_train_raw, y_train_raw, test_size=0.2, random_state=0)

In [20]:
# Initialising the CNN
classifier = ResNet50(include_top=False, weights='imagenet', 
                   input_shape=(img_width,img_height, 3))

#Add a layer where input is the output of the  second last layer 
x = Flatten(name='flatten')(classifier.output)
#x = Dense(num_classes, activation='softmax', name='predictions')(x)
x = BatchNormalization()(x)
x = Dropout(0.8)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.8)(x)
x = Dense(120, activation='softmax')(x)

#Then create the corresponding model 
my_model = Model(inputs=classifier.input, outputs=x)
my_model.summary()





#classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
my_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

ValueError: Input size must be at least 197x197; got `input_shape=(128, 128, 3)`

In [ ]:
my_model.fit(X_train, Y_train, epochs=20,batch_size=bs, validation_data=(X_val, Y_val), verbose=1)